In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)


In [2]:
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [3]:
#Total Number of Players

uniqueplayer = purchase_data["SN"].unique()
uniqueplayer_count = len(uniqueplayer)
uniqueplayer_df = pd.DataFrame({"Total Players" : [uniqueplayer_count]})
uniqueplayer_df

,Total Players
0,576


In [4]:
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

uniqueitems = purchase_data["Item ID"].nunique()
avgprice = purchase_data["Price"].mean()
totalpurchases = purchase_data["Purchase ID"].count()
sumrevenue = purchase_data["Price"].sum()

purchase_summary_df = pd.DataFrame({"Number of Unique Items" : [uniqueitems],
                                   "Average Purchase Price" : ["${:,.2f}".format(avgprice)],
                                   "Total Number of Purchases" : [totalpurchases],
                                   "Total Revenue" : ["${:,.2f}".format(sumrevenue)]})
purchase_summary_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [27]:
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

unique_gen_df = purchase_data.drop_duplicates(subset = "SN", keep = "first", inplace = False)

wholecount = unique_gen_df["Gender"].count()
unique_df = unique_gen_df.groupby(["Gender"]).count()

unique_df["Percentage of Players"] = (unique_df["Purchase ID"] / wholecount) * 100
def format(x):
        return "{:.2f}%".format(x)
unique_df["Percentage of Players"] = unique_df["Percentage of Players"].apply(format)

unique_df = unique_df.drop(columns={"SN","Age","Item ID","Item Name","Price"})
unique_df = unique_df.rename(columns={"Purchase ID":"Total Count of Players"})
unique_df = unique_df.sort_values("Total Count of Players", ascending=False)

unique_df

,Total Count of Players,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [60]:
# The below each broken by gender
gender_needed = pd.DataFrame(purchase_data, columns=["SN","Gender","Purchase ID","Price"])

# Purchase Count
purcount_gen = gender_needed.groupby(["Gender"]).count()
# Average Purchase Price
avgprice_gen = gender_needed.groupby(["Gender"]).mean()
# Total Purchase Value
tvp_gen = gender_needed.groupby(["Gender"]).sum()

merge_gen = pd.DataFrame.merge(purcount_gen, avgprice_gen, on="Gender",suffixes=("_count","_avg"))
merge_gen2 = pd.DataFrame.merge(merge_gen, tvp_gen, on="Gender",suffixes=("_merge","_tvp"))

merge_gen2 = merge_gen2.drop(columns={"Purchase ID_count","Price_count","Purchase ID_avg","Purchase ID"})
merge_gen2 = merge_gen2.rename(columns={"SN":"Purchase Count","Price_avg":"Average Purchase Price","Price":"Total Purchase Value"})
merge_gen3 = pd.DataFrame.merge(merge_gen2, unique_df, on="Gender")

# Average Purchase Total per Person by Gender
merge_gen3["Avg Total Purchase per Person"] = (merge_gen3["Total Purchase Value"] / merge_gen3["Total Count of Players"])
merge_gen3 = merge_gen3.drop(columns={"Total Count of Players"})

def format(x):
        return "${:.2f}".format(x)
merge_gen3["Average Purchase Price"] = merge_gen3["Average Purchase Price"].apply(format)
merge_gen3["Total Purchase Value"] = merge_gen3["Total Purchase Value"].apply(format)
merge_gen3["Avg Total Purchase per Person"] = merge_gen3["Avg Total Purchase per Person"].apply(format)

merge_gen3 = merge_gen3.drop(columns={"Percentage of Players"})

merge_gen3


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [70]:
a = purchase_data["Age"].max()
b = purchase_data["Age"].min()
a , b

(45, 7)

In [66]:
unique_age_df = purchase_data.drop_duplicates(subset = "SN", keep = "first", inplace = False)

great = unique_age_df["Age"].max()
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, great]

group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]

unique_age_df["Age Bracket"] = pd.cut(unique_age_df["Age"], bins, labels=group_names, include_lowest=True)
wholecount = unique_age_df["Age"].count()

unique_age_df = unique_age_df.groupby(["Age Bracket"]).count()

unique_age_df["Percentage of Players"] = (unique_age_df["Purchase ID"] / wholecount) * 100
def format(x):
        return "{:.2f}%".format(x)
unique_age_df["Percentage of Players"] = unique_age_df["Percentage of Players"].apply(format)

unique_age_df = unique_age_df.drop(columns={"SN","Age","Item ID","Item Name","Price","Gender"})
unique_age_df = unique_age_df.rename(columns={"Purchase ID":"Total Count of Players"})

unique_age_df

/Users/ashtonsmith/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Total Count of Players,Percentage of Players
Age Bracket,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [72]:
# The below each broken by gender
age_needed = pd.DataFrame(purchase_data, columns=["SN","Purchase ID","Price","Age"])

great = age_needed["Age"].max()
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, great]

group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]

age_needed["Age Bracket"] = pd.cut(age_needed["Age"], bins, labels=group_names, include_lowest=True)

# Purchase Count
purcount_age = age_needed.groupby(["Age Bracket"]).count()
# Average Purchase Price
avgprice_age = age_needed.groupby(["Age Bracket"]).mean()
# Total Purchase Value
tvp_age = age_needed.groupby(["Age Bracket"]).sum()

merge_age = pd.DataFrame.merge(purcount_age, avgprice_age, on="Age Bracket",suffixes=("_count","_avg"))
merge_age2 = pd.DataFrame.merge(merge_age, tvp_age, on="Age Bracket",suffixes=("_merge","_tvp"))

merge_age2 = merge_age2.drop(columns={"Purchase ID_count","Price_count","Purchase ID_avg","Purchase ID"})
merge_age2 = merge_age2.rename(columns={"SN":"Purchase Count","Price_avg":"Average Purchase Price","Price":"Total Purchase Value"})
merge_age3 = pd.DataFrame.merge(merge_age2, unique_age_df, on="Age Bracket")

# Average Purchase Total per Person by Gender
merge_age3["Avg Total Purchase per Person"] = (merge_age3["Total Purchase Value"] / merge_age3["Total Count of Players"])
merge_age3 = merge_age3.drop(columns={"Total Count of Players","Percentage of Players","Age","Age_avg","Age_count"})

def format(x):
        return "${:.2f}".format(x)
merge_age3["Average Purchase Price"] = merge_age3["Average Purchase Price"].apply(format)
merge_age3["Total Purchase Value"] = merge_age3["Total Purchase Value"].apply(format)
merge_age3["Avg Total Purchase per Person"] = merge_age3["Avg Total Purchase per Person"].apply(format)

merge_age3

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bracket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [184]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
player_needitem_df = pd.DataFrame(purchase_data, columns=["SN","Purchase ID","Price"])

player_purchase_count = player_needitem_df.groupby("SN").count()
player_tvp = player_needitem_df.groupby("SN").sum()
player_average = player_needitem_df.groupby("SN").mean()

player_merge1 = pd.DataFrame.merge(player_purchase_count, player_tvp, on="SN", suffixes=("_count","_tvp"))
player_merge2 = pd.DataFrame.merge(player_merge1, player_average, on="SN", suffixes=("","_average"))
player_merge2 = player_merge2.drop(columns={"Purchase ID_tvp","Price_count","Purchase ID"})
player_merge2 = player_merge2.rename(columns={"Purchase ID_count":"Purchase Count","Price":"Average Purchase Price","Price_tvp":"Total Purchase Value"})

player_merge2 = player_merge2.sort_values("Total Purchase Value", ascending=False)

player_merge2 = player_merge2[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

def format(x):
        return "${:.2f}".format(x)
player_merge2['Average Purchase Price'] = player_merge2['Average Purchase Price'].apply(format)
player_merge2['Total Purchase Value'] = player_merge2['Total Purchase Value'].apply(format)

player_merge2

# SN
# Purchase Count
# Average Purchase Price
# Total Purchase Value



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [222]:
# Identify the 5 most popular items by purchase count, then list (in a table):
popneeditem_df = pd.DataFrame(purchase_data, columns=["Item ID","Item Name","Price"])

pop_tvp = popneeditem_df.groupby(["Item ID","Item Name"]).sum()
pop_item_purcount = popneeditem_df.groupby(["Item ID","Item Name"]).count()
pop_itemprice = popneeditem_df.groupby(["Item ID","Item Name"]).mean()

pop_merge1 = pd.DataFrame.merge(pop_tvp, pop_item_purcount, on=["Item ID","Item Name"])
pop_merge2 = pd.DataFrame.merge(pop_merge1, pop_itemprice, on=["Item ID","Item Name"])
pop_merge2 = pop_merge2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Purchase Count","Price":"Item Price"})
pop_merge2 = pop_merge2[["Purchase Count","Item Price", "Total Purchase Value"]]

pop_merge2 = pop_merge2.sort_values("Purchase Count", ascending=False)

def format(x):
    return "${:.2f}".format(x)

pop_merge2['Total Purchase Value'] = pop_merge2['Total Purchase Value'].apply(format)
pop_merge2['Item Price'] = pop_merge2['Item Price'].apply(format)

pop_merge2.head(5)


# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [220]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):
needitem_df = pd.DataFrame(purchase_data, columns=["Item ID","Item Name","Price"])

tvp = needitem_df.groupby(["Item ID","Item Name"]).sum()
item_purcount = needitem_df.groupby(["Item ID","Item Name"]).count()
itemprice = needitem_df.groupby(["Item ID","Item Name"]).mean()

merge1 = pd.DataFrame.merge(tvp, item_purcount, on=["Item ID","Item Name"])
merge2 = pd.DataFrame.merge(merge1, itemprice, on=["Item ID","Item Name"])
merge2 = merge2.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Purchase Count","Price":"Item Price"})
merge2 = merge2[["Purchase Count","Item Price", "Total Purchase Value"]]

merge2 = merge2.sort_values("Total Purchase Value", ascending=False)

def format(x):
    return "${:.2f}".format(x)

merge2['Total Purchase Value'] = merge2['Total Purchase Value'].apply(format)
merge2['Item Price'] = merge2['Item Price'].apply(format)

merge2.head(5)


# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
